# Video Game Dataset Scraper from Metacritic (2000–2025)

*Based on the code from the repository [projectGames](https://github.com/BrunoBVR/projectGames) (2020), but rewritten to account for the current structure of the Metacritic website and to extract the specific data I want to collect.*

Using Metacritic’s own search tool ([Metacritic](https://www.metacritic.com/browse/games/score/metascore/all/all/filtered?sort=desc)), data was scraped to build a large dataframe containing ratings, release dates, genres, developers, and publishers of games (from 2000 to 2025) across all platforms.

Features:
- httpx + asyncio with concurrency limit.  
- Extraction of listings and details (JSON-LD + platforms + scores).  
- Incremental saving per page in `pages/`.  
- Consolidated DataFrame in `metacritic_dataset.csv`.  

**Data scraped on October 7, 2025.**


## Initial requirements and parameters

In [1]:
%pip install httpx beautifulsoup4 lxml pandas

  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached beautifulsoup4-4.14.2-py3-none-any.whl.metadata (3.8 kB)
  Using cached lxml-6.0.2-cp312-cp312-win_amd64.whl.metadata (3.7 kB)
  Using cached anyio-4.11.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached certifi-2025.10.5-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached soupsieve-2.8-py3-none-any.whl.metadata (4.6 kB)
  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached numpy-2.3.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Asynchronous Metacritic scraper (pages range configurable)
import asyncio, random, json, time, re
from pathlib import Path
from typing import List, Dict, Any
import httpx
import pandas as pd
from bs4 import BeautifulSoup

Parameters you can adjust within the code:
- `PAGES`: Total pages of games in the Metacritic Web. There are 580 as of October 2025.  
- `MAX_CONCURRENCY`: Number of workers. Increase/decrease depending on speed vs. blocking risk.   
- `OUTPUT_DIR`: Path for storing the pages csvs

In [ ]:
BASE_DOMAIN = "https://www.metacritic.com"
LIST_URL = ("https://www.metacritic.com/browse/game/?releaseYearMin=1958&"
            "releaseYearMax=2025&page={page}")   # List of all games

PAGES = 580  # adjust for more pages
MAX_CONCURRENCY = 15
OUTPUT_DIR = Path("pages")
OUTPUT_DIR.mkdir(exist_ok=True)

HEADERS = {
    "User-Agent": ("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                    "(KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36"),
    "Accept-Language": "en-US,en;q=0.9"
}

## Network helper: concurrency & robust fetching

This cell defines the global semaphore and the `fetch(client, url, ...)` async helper used across the scraper.

- Purpose: perform HTTP GETs with controlled concurrency, retries, exponential backoff and small random jitter to reduce rate-limiting and server load.
- Key behaviors:
  - Uses `MAX_CONCURRENCY` via an `asyncio.Semaphore` to limit concurrent requests.
  - Retries on network errors and on 403/429 responses (rate limits), using `backoff ** attempt` with jitter.
  - Returns the response body (`str`) on HTTP 200; returns `None` on non-retryable status codes or if all retries fail.
  - Adds a tiny random sleep after successful requests to avoid immediate bursts.

In [ ]:
sem = asyncio.Semaphore(MAX_CONCURRENCY)

async def fetch(client: httpx.AsyncClient, url: str, retries: int = 3, backoff: float = 1.5) -> str | None:
    for attempt in range(retries):
        try:
            async with sem:
                r = await client.get(url, timeout=25)
        except httpx.RequestError:
            await asyncio.sleep(backoff ** attempt + random.uniform(0, 0.5))
            continue
        if r.status_code == 200:
            # Small jitter to avoid hammering the server
            await asyncio.sleep(random.uniform(0.05, 0.15))
            return r.text
        if r.status_code in (403, 429):
            await asyncio.sleep(backoff ** attempt + random.uniform(0, 1))
            continue
        # Other status codes: do not retry
        return None
    return None

NameError: name 'MAX_CONCURRENCY' is not defined

## Parsing helpers 

### List-page parser

This cell defines `parse_list_page(html)`.

- Purpose: parse a Metacritic listing/search results page and extract every game entry found on that page, returning a list of dictionaries like `[{"name": <game name>, "url": <absolute url>}, ...]`.
- Key behavior: selects card elements with the `.c-finderProductCard` selector, extracts the title and anchor, strips leading numeric prefixes from names, converts relative links to absolute using `BASE_DOMAIN`, and skips malformed cards.
- Note: uses BeautifulSoup (`lxml`) and `re`; selectors may need updating if Metacritic changes its markup.

In [ ]:
def parse_list_page(html: str) -> List[Dict[str, str]]:
    soup = BeautifulSoup(html, 'lxml')
    cards = soup.select('.c-finderProductCard')
    games = []
    for c in cards:
        title = c.select_one('.c-finderProductCard_titleHeading')
        if not title:
            continue
        # Clean numbering at the start if present
        name = re.sub(r'^\d+\.\s*', '', title.get_text(strip=True))
        link = c.select_one('a.c-finderProductCard_container')
        if not link:
            continue
        href = link.get('href')
        if href.startswith('/'):
            href = BASE_DOMAIN + href
        games.append({'name': name, 'url': href})
    return games

### JSON-LD and developer extraction

This cell provides two small parsing helpers used by the detail-page processor.

- `extract_json_ld(soup) -> dict` — looks for the page's structured JSON-LD (`<script type="application/ld+json">`) and returns the parsed object. If the script is missing or invalid it returns `{}`.
- `extract_developer(soup) -> str | None` — extracts the first developer name using the `div.c-gameDetails_Developer ul li` selector and returns `None` if not found.

Notes:
- Both helpers expect a BeautifulSoup object (`soup`) created with the `lxml` parser.
- If Metacritic changes markup, update the selectors or add extra fallbacks.

In [ ]:
def extract_json_ld(soup: BeautifulSoup) -> Dict[str, Any]:
    script = soup.find('script', attrs={"type": "application/ld+json"})
    if not script or not script.string:
        return {}
    try:
        return json.loads(script.string)
    except json.JSONDecodeError:
        return {}

def extract_developer(soup: BeautifulSoup) -> str | None:
    dev_li = soup.select_one("div.c-gameDetails_Developer ul li")
    return dev_li.get_text(strip=True) if dev_li else None

### Platform scores and user-score extractor

This cell defines helpers that extract platform-specific metascores and user-score pages from a game's detail HTML.

- `extract_platform_scores(soup)`
  - Input: BeautifulSoup object for a game detail page.
  - Output: `(metascores, user_urls)` where `metascores` is a list of per-platform metascore strings (or None) and `user_urls` is a list of absolute URLs to the user-review pages for platforms that expose critic/user review pages.
  - Behavior: finds platform tiles via `.c-gamePlatformsSection_list a.c-gamePlatformTile`, reads the platform name and metascore, normalizes relative hrefs to absolute using `BASE_DOMAIN`, and builds user-review URLs by swapping `critic-reviews` to `user-reviews` when appropriate.

- `extract_user_score(html)`
  - Input: raw HTML (string) fetched from a user-reviews page.
  - Output: the user score as a string, or `None` if not found.

Note:
- If Metacritic's markup changes, update the CSS selectors and the `critic-reviews` -> `user-reviews` heuristic.

In [ ]:
def extract_platform_scores(soup: BeautifulSoup):
    platform_blocks = soup.select('.c-gamePlatformsSection_list a.c-gamePlatformTile')
    platforms, metascores, user_urls = [], [], []
    for a in platform_blocks:
        title_tag = a.find('title')
        platform_name = title_tag.text.strip() if title_tag else None
        score_span = a.select_one('.c-siteReviewScore span')
        meta_val = score_span.text.strip() if score_span else None
        href = a.get('href', '')
        if href.startswith('/'):
            href_full = BASE_DOMAIN + href
        else:
            href_full = href
        if 'critic-reviews/?platform=' in href_full:
            user_urls.append(href_full.replace('critic-reviews', 'user-reviews'))
        platforms.append(platform_name)
        metascores.append(meta_val)
    return metascores, user_urls

def extract_user_score(html: str) -> str | None:
    soup = BeautifulSoup(html, 'lxml')
    container = soup.find(attrs={"data-testid": "score-card-overview"})
    if not container:
        return None
    span = container.select_one('div.c-siteReviewScore span')
    return span.get_text(strip=True) if span else None

## Game detail processing

This cell implements `get_game_details(client, game)` which fetches a game's detail page and returns one or more data rows ready to be placed into the dataset.

- Input: `client` (an `httpx.AsyncClient`) and `game` (a dict with `{"name": ..., "url": ...}` produced by `parse_list_page`).
- Output: A list of dictionaries, one per platform (or a single dict with `platform=None` if platforms aren't found). Each dict contains keys: `name`, `platform`, `release_date`, `metascore`, `user_score`, `developer`, `publisher`, `genre`.
- Behavior summary:
  - Fetch detail HTML via `fetch` and parse with BeautifulSoup.
  - Prefer JSON-LD (`extract_json_ld`) for canonical fields (platforms, release date, publishers, genres).
  - Extract per-platform metascores and user-review URLs via `extract_platform_scores`, then fetch user pages in parallel and extract user scores with `extract_user_score`.
  - Assemble rows: if JSON-LD lists platforms, create one row per platform; otherwise produce a single row with `platform=None`.

Edge cases & notes:
- Missing or invalid JSON-LD will fall back to DOM-extracted values where available.
- Network failures on user-score pages yield `None` for those scores.
- This function is intentionally tolerant: it returns an empty list when the main page fetch fails, so the orchestration can continue.

In [ ]:
async def get_game_details(client: httpx.AsyncClient, game: Dict[str, str]) -> List[Dict[str, Any]]:
    html = await fetch(client, game['url'])
    if not html:
        return []
    soup = BeautifulSoup(html, 'lxml')
    data_ld = extract_json_ld(soup)

    metascores, user_urls = extract_platform_scores(soup)

    platforms = data_ld.get("gamePlatform", [])
    release_date = data_ld.get('datePublished')
    publisher_raw = data_ld.get('publisher', [])
    if isinstance(publisher_raw, dict):
        publishers = [publisher_raw.get('name')]
    elif isinstance(publisher_raw, list):
        publishers = [p.get('name') if isinstance(p, dict) else p for p in publisher_raw]
    else:
        publishers = [publisher_raw] if publisher_raw else []

    genres = data_ld.get('genre', [])
    if isinstance(genres, str):
        genres = [genres]

    developer = extract_developer(soup)

    user_scores = []
    if user_urls:
        # Parallelize user score fetches per platform
        tasks = [fetch(client, u) for u in user_urls]
        user_pages = await asyncio.gather(*tasks)
        for page_html in user_pages:
            if not page_html:
                user_scores.append(None)
            else:
                user_scores.append(extract_user_score(page_html))
    else:
        user_scores = [None] * len(platforms)

    rows = []
    if not platforms:
        rows.append({
            'name': game['name'],
            'platform': None,
            'release_date': release_date,
            'metascore': None,
            'user_score': None,
            'developer': developer,
            'publisher': publishers,
            'genre': genres
        })
    else:
        for i, plat in enumerate(platforms):
            rows.append({
                'name': game['name'],
                'platform': plat,
                'release_date': release_date,
                'metascore': metascores[i] if i < len(metascores) else None,
                'user_score': user_scores[i] if i < len(user_scores) else None,
                'developer': developer,
                'publisher': publishers,
                'genre': genres
            })
    return rows

## Page orchestration

This cell contains the `process_page(client, page_number)` helper that orchestrates scraping for a single listing page and writes the corresponding csvs.

The code fetches a listing page, then parses the list of games, then fetches details for each game concurrently, and return a pandas.DataFrame with the rows for that page.

Notes:
- The function is tolerant: a failed listing fetch yields an empty DataFrame so the main orchestrator can continue.
- Per-page CSVs enable resuming or partial runs without re-fetching already-saved pages.

In [ ]:
async def process_page(client: httpx.AsyncClient, page_number: int) -> pd.DataFrame:
    list_html = await fetch(client, LIST_URL.format(page=page_number))
    if not list_html:
        print(f"[WARN] Failed to fetch listing page {page_number}")
        return pd.DataFrame()
    games = parse_list_page(list_html)
    print(f"Page {page_number}: {len(games)} games detected")

    # Process games concurrently
    tasks = [get_game_details(client, g) for g in games]
    results = await asyncio.gather(*tasks)

    flat_rows = [row for game_rows in results for row in game_rows]
    df_page = pd.DataFrame(flat_rows)

    # Incremental save per page (keep individual page CSVs)
    if not df_page.empty:
        out_file = OUTPUT_DIR / f"games_data-page{page_number}.csv"
        df_page.to_csv(out_file, index=False, encoding='utf-8')
        print(f"Saved page {page_number} -> {len(df_page)} rows")
    else:
        print(f"Page {page_number} has no valid rows")
    return df_page

## Main async runner

This cell declares `main_async(batch_size)`, the high-level orchestrator that kicks off page scraping using batches to optimize processing. Tune `BATCH_SIZE`, `PAGES` and `MAX_CONCURRENCY` to balance throughput and rate-limit risk.

In [ ]:
async def main_async(batch_size: int = 100):
    async with httpx.AsyncClient(headers=HEADERS, follow_redirects=True, timeout=30.0) as client:
        for start in range(1, PAGES + 1, batch_size):
            end = min(start + batch_size - 1, PAGES)
            page_range = list(range(start, end + 1))
            print(f"Processing pages {start} to {end} (batch size {batch_size})")
            tasks = [process_page(client, p) for p in page_range]
            page_dfs = await asyncio.gather(*tasks)
            # page_dfs are saved per page inside process_page (incremental),
            # so here we just log totals and free memory.
            total_rows = sum(len(df) for df in page_dfs if hasattr(df, 'shape'))
            print(f"Chunk {start}-{end}: fetched {len(page_dfs)} pages -> {total_rows} rows")
            # Explicitly drop references and run GC to free memory between batches
            del page_dfs
            import gc
            gc.collect()
            # Short pause between batches to reduce burstiness
            await asyncio.sleep(0.2)

    print("All batches processed")
    return

print("Async functions defined. Run the next cell to start scraping (it will process pages in batches).")

In [ ]:
# Execute async scraping (batched)
import pandas as pd, asyncio

BATCH_SIZE = 100  # adjust this value as needed (e.g., 50, 100)
try:
    asyncio.run(main_async(batch_size=BATCH_SIZE))
except RuntimeError:
    # Jupyter notebook compatibility
    import nest_asyncio, asyncio as _asyncio
    nest_asyncio.apply()
    _asyncio.get_event_loop().run_until_complete(main_async(batch_size=BATCH_SIZE))

Página 510: 24 juegos detectados
Página 512: 24 juegos detectados
Página 506: 24 juegos detectados
Página 513: 24 juegos detectados
Página 507: 24 juegos detectados
Página 515: 24 juegos detectados
Página 509: 24 juegos detectados
Página 505: 24 juegos detectados
Página 504: 24 juegos detectados
Página 508: 24 juegos detectados
Página 503: 24 juegos detectados
Página 501: 24 juegos detectados
Página 511: 24 juegos detectados
Página 514: 24 juegos detectados
Página 502: 24 juegos detectados
Página 516: 24 juegos detectados
Página 519: 24 juegos detectados
Página 517: 24 juegos detectados
Página 518: 24 juegos detectados
Página 520: 24 juegos detectados
Página 521: 24 juegos detectados
Página 522: 24 juegos detectados
Página 530: 24 juegos detectados
Página 528: 24 juegos detectados
Página 525: 24 juegos detectados
Página 527: 24 juegos detectados
Página 531: 24 juegos detectados
Página 529: 24 juegos detectados
Página 523: 24 juegos detectados
Página 524: 24 juegos detectados
Página 533

,name,platform,release_date,metascore,user_score,developer,publisher,genre
0,"12,001.Spy Hunter 2",PlayStation 2,2003-11-24,57.0,NaN,Angel Studios,[Midway],[Auto Racing]
1,"12,001.Spy Hunter 2",Xbox,2003-11-24,57.0,NaN,Angel Studios,[Midway],[Auto Racing]
2,"12,001.Spy Hunter 2",GameCube,2003-11-24,NaN,NaN,Angel Studios,[Midway],[Auto Racing]
3,"12,002.Mobile Suit Gundam: Gundam vs. Zeta Gundam",GameCube,2005-06-28,57.0,7.9,Capcom,"[Bandai America Inc., Bandai]",[Vehicle Combat Sim]
4,"12,002.Mobile Suit Gundam: Gundam vs. Zeta Gundam",PlayStation 2,2005-06-28,NaN,NaN,Capcom,"[Bandai America Inc., Bandai]",[Vehicle Combat Sim]
5,"12,003.Mobile Suit Gundam Seed: Battle Assault",Game Boy Advance,2004-08-10,57.0,6.0,Natsume,[Bandai],[2D Fighting]
6,"12,004.The LEGO Movie 2 Videogame",Nintendo Switch,2019-02-26,56.0,5.2,TT Games,[Warner Bros. Interactive Entertainment],[Action Adventure]
7,"12,004.The LEGO Movie 2 Videogame",PlayStation 4,2019-02-26,57.0,3.7,TT Games,[Warner Bros. Interactive Entertainment],[Action Adventure]
8,"12,004.The LEGO Movie 2 Videogame",Xbox One,2019-02-26,65.0,5.5,TT Games,[Warner Bros. Interactive Entertainment],[Action Adventure]
9,"12,004.The LEGO Movie 2 Videogame",PC,2019-02-26,56.0,5.8,TT Games,[Warner Bros. Interactive Entertainment],[Action Adventure]


,name,platform,release_date,metascore,user_score,developer,publisher,genre
4204,"13,908.Double Dragon II: Wander of the Dragons",Xbox 360,2013-04-05,17.0,1.6,Gravity,[CFK],[2D Beat-'Em-Up]
4205,"13,909.Vroom in the Night Sky",Nintendo Switch,2017-04-05,17.0,3.3,Poisoft,[Poisoft],[Biking]
4206,"13,910.Yaris",Xbox 360,2007-10-10,17.0,5.2,Backbone Entertainment,[Backbone Entertainment],[Auto Racing]
4207,"13,911.Family Party: 30 Great Games Obstacle A...",Wii U,2012-12-04,11.0,2.8,"Art Co., Ltd.","[D3Publisher, Namco Bandai Games]",[Party]
4208,"13,912.Stormgate",PC,2025-08-05,NaN,4.3,Frost Giant Studios,[Frost Giant Studios],[Command RTS]


### Concatenate all pages csvs

In [ ]:
import pandas as pd
# Create a list of pages dataframes
pages = [pd.read_csv(f'pages/games_data-page{p}.csv') for p in range(1, PAGES + 1)]
# Concatenate all dataframes into one
df_ultimate = pd.concat(pages, ignore_index=True).reset_index(drop=True)

In [ ]:
# Fix names of games with numbers at the start
df_ultimate['name'] = df_ultimate['name'].str.strip().replace(r'^\d{1,3}(,\d{3})+\.\s*', '', regex=True)

# The Dataset

The columns are:
* **name**: Name of the game  
* **platform**: Platform on which the game was released  
* **release_date**: The date the game was released (first release date on any platform; does not include separate dates for re-releases on other consoles)  
* **metascore**: Metascore, the average rating given by Metacritic based on reviews from professional critics  
* **user score**: Average rating given by Metacritic users  
* **developer**: Game developer  
* **publisher**: Game publisher(s)  
* **genre**: Game genre(s), may include mixed genres  

In [ ]:
df_ultimate.head(15)

,name,platform,release_date,metascore,user_score,developer,publisher,genre
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,99,9.1,Nintendo,"['Nintendo', 'Gradiente']",['Open-World Action']
1,SoulCalibur,Dreamcast,1999-09-08,98,7.6,Namco,['Namco'],['3D Fighting']
2,SoulCalibur,iOS (iPhone/iPad),1999-09-08,73,7.8,Namco,['Namco'],['3D Fighting']
3,SoulCalibur,Xbox 360,1999-09-08,79,7.3,Namco,['Namco'],['3D Fighting']
4,Grand Theft Auto IV,PlayStation 3,2008-04-29,98,8.0,Rockstar North,"['Rockstar Games', 'Capcom']",['Open-World Action']
5,Grand Theft Auto IV,Xbox 360,2008-04-29,98,8.3,Rockstar North,"['Rockstar Games', 'Capcom']",['Open-World Action']
6,Grand Theft Auto IV,PC,2008-04-29,90,7.8,Rockstar North,"['Rockstar Games', 'Capcom']",['Open-World Action']
7,Super Mario Galaxy,Wii,2007-11-12,97,9.1,Nintendo,['Nintendo'],['3D Platformer']
8,Super Mario Galaxy,Nintendo Switch,2007-11-12,tbd,NaN,Nintendo,['Nintendo'],['3D Platformer']
9,Super Mario Galaxy 2,Wii,2010-05-23,97,9.0,Nintendo EAD Tokyo,"['Nintendo', 'iQue']",['3D Platformer']


In [ ]:
df_ultimate.tail(15)

,name,platform,release_date,metascore,user_score,developer,publisher,genre
36887,"13,904.Deal or No Deal",PC,2007-07-23,tbd,NaN,Artefacts Studio,"['Destination Software', 'Mindscape', 'Koch Me...",['Trivia']
36888,"13,904.Deal or No Deal",DS,2007-07-23,20,3.8,Artefacts Studio,"['Destination Software', 'Mindscape', 'Koch Me...",['Trivia']
36889,"13,904.Deal or No Deal",Game Boy Advance,2007-07-23,56,5.0,Artefacts Studio,"['Destination Software', 'Mindscape', 'Koch Me...",['Trivia']
36890,"13,904.Deal or No Deal",Wii,2007-07-23,NaN,NaN,Artefacts Studio,"['Destination Software', 'Mindscape', 'Koch Me...",['Trivia']
36891,"13,905.Alone in the Dark: Illumination",PC,2015-06-11,19,1.3,Pure FPS,['Atari SA'],['Survival']
36892,"13,906.Ride to Hell: Retribution",PC,2013-06-25,16,1.4,Eutechnyx,['Deep Silver'],['Action Adventure']
36893,"13,906.Ride to Hell: Retribution",PlayStation 3,2013-06-25,13,1.1,Eutechnyx,['Deep Silver'],['Action Adventure']
36894,"13,906.Ride to Hell: Retribution",Xbox 360,2013-06-25,19,1.3,Eutechnyx,['Deep Silver'],['Action Adventure']
36895,"13,907.SPOGS Racing",Wii,2008-07-07,18,6.3,Pronto Games,['D2C Games'],['Auto Racing']
36896,"13,907.SPOGS Racing",PC,2008-07-07,NaN,NaN,Pronto Games,['D2C Games'],['Auto Racing']


In [ ]:
# Explore the dataframe
df_ultimate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36902 entries, 0 to 36901
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          36902 non-null  object
 1   platform      36902 non-null  object
 2   release_date  36859 non-null  object
 3   metascore     31170 non-null  object
 4   user_score    24556 non-null  object
 5   developer     36891 non-null  object
 6   publisher     36902 non-null  object
 7   genre         36902 non-null  object
dtypes: object(8)
memory usage: 2.3+ MB


In [ ]:
# Save the raw dataframe to CSV
df_ultimate.to_csv('metacritic_dataset_raw.csv', index=False)

## Data Cleaning

In [ ]:
# Examine cases with null columns
df_ultimate[df_ultimate.isnull().any(axis=1)].head(15)

,name,platform,release_date,metascore,user_score,developer,publisher,genre
8,Super Mario Galaxy,Nintendo Switch,2007-11-12,tbd,NaN,Nintendo,['Nintendo'],['3D Platformer']
10,Super Mario Galaxy 2,Nintendo Switch,2010-05-23,tbd,NaN,Nintendo EAD Tokyo,"['Nintendo', 'iQue']",['3D Platformer']
21,Tony Hawk's Pro Skater 3,Nintendo 64,2001-10-30,NaN,NaN,Neversoft Entertainment,"['Activision', 'Success']",['Skating']
33,Metroid Prime,Wii,2002-11-17,NaN,NaN,Retro Studios,['Nintendo'],['FPS']
37,Grand Theft Auto III,Xbox,2001-10-22,NaN,NaN,DMA Design,"['Rockstar Games', 'SCEA', 'Capcom', 'Take-Two...",['Open-World Action']
38,Grand Theft Auto III,PlayStation 4,2001-10-22,NaN,NaN,DMA Design,"['Rockstar Games', 'SCEA', 'Capcom', 'Take-Two...",['Open-World Action']
39,Grand Theft Auto III,Game Boy Advance,2001-10-22,NaN,NaN,DMA Design,"['Rockstar Games', 'SCEA', 'Capcom', 'Take-Two...",['Open-World Action']
52,GoldenEye 007,Xbox One,1997-08-25,NaN,NaN,Rare Ltd.,"['Nintendo', 'Gradiente']",['FPS']
53,GoldenEye 007,Xbox Series X,1997-08-25,NaN,NaN,Rare Ltd.,"['Nintendo', 'Gradiente']",['FPS']
66,Tekken 3,Dreamcast,1998-04-29,NaN,NaN,Namco,"['Namco', 'SCEE']",['3D Fighting']


In [ ]:
# Examine cases with Metascore 'tbd', which is the same as no score
df_ultimate[df_ultimate['metascore'] == 'tbd']

,name,platform,release_date,metascore,user_score,developer,publisher,genre
8,Super Mario Galaxy,Nintendo Switch,2007-11-12,tbd,NaN,Nintendo,['Nintendo'],['3D Platformer']
10,Super Mario Galaxy 2,Nintendo Switch,2010-05-23,tbd,NaN,Nintendo EAD Tokyo,"['Nintendo', 'iQue']",['3D Platformer']
36,Grand Theft Auto III,iOS (iPhone/iPad),2001-10-22,tbd,7.3,DMA Design,"['Rockstar Games', 'SCEA', 'Capcom', 'Take-Two...",['Open-World Action']
51,GoldenEye 007,Xbox 360,1997-08-25,tbd,6.4,Rare Ltd.,"['Nintendo', 'Gradiente']",['FPS']
70,The House in Fata Morgana - Dreams of the Reve...,PlayStation 4,2021-04-09,tbd,7.8,HuneX,"['Limited Run Games', 'dramatic create', 'HuneX']",['Visual Novel']
...,...,...,...,...,...,...,...,...
36879,Fast & Furious: Showdown,3DS,2013-05-21,tbd,2.0,Firebrand Games,['Activision'],['Auto Racing']
36881,Drake of the 99 Dragons,PC,2003-11-03,tbd,2.0,Idol FX,['Majesco'],['Third Person Shooter']
36884,Afro Samurai 2: Revenge of Kuma Volume One,Xbox One,2015-09-22,tbd,2.6,Versus Evil,['Versus Evil'],"[""3D Beat-'Em-Up""]"
36887,Deal or No Deal,PC,2007-07-23,tbd,NaN,Artefacts Studio,"['Destination Software', 'Mindscape', 'Koch Me...",['Trivia']


In [ ]:
# Examine cases with user_score 'tbd', which is the same as no score
df_ultimate[df_ultimate['user_score'] == 'tbd']

,name,platform,release_date,metascore,user_score,developer,publisher,genre
16505,Coridden,PC,2025-01-29,75,tbd,Aftnareld,['Anshar Studios'],['Action RPG']
17013,Pinball FX,PlayStation 4,2023-02-16,tbd,tbd,Zen Studios,['Zen Studios'],['Pinball']
20792,Marvel's Guardians of the Galaxy - Episode 2: ...,iOS (iPhone/iPad),2017-06-06,tbd,tbd,Telltale Games,['Telltale Games'],['Third-Person Adventure']
36424,Pimp My Ride,Wii,2006-12-06,26,tbd,Eutechnyx,['Activision'],['Arcade Racing']


In [ ]:
# Delete null rows
df_clean = df_ultimate.dropna(subset=['name', 'platform', 'release_date', 'metascore', 'user_score', 'developer', 'publisher', 'genre']).reset_index(drop=True)

In [ ]:
# Delete metascore 'tbd' rows
df_clean = df_clean[df_clean['metascore'] != 'tbd'].reset_index(drop=True)

In [ ]:
# Delete user_score 'tbd' rows
df_clean = df_clean[df_clean['user_score'] != 'tbd'].reset_index(drop=True)

In [ ]:
# Convert name column to string
df_clean["name"] = df_clean["name"].apply(str)

In [ ]:
# Convertir publisher column to list
import ast

def safe_list(val):
	if isinstance(val, list):
		return val
	if isinstance(val, str) and val.startswith('['):
		try:
			return ast.literal_eval(val)
		except Exception:
			return []
	return [val] if isinstance(val, str) else []

df_clean['publisher'] = df_clean['publisher'].apply(safe_list)

In [ ]:
# Convert release_date to datetime
df_clean['release_date'] = pd.to_datetime(df_clean['release_date'], errors='coerce')

In [ ]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22264 entries, 0 to 22263
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          22264 non-null  object
 1   platform      22264 non-null  object
 2   release_date  22264 non-null  object
 3   metascore     22264 non-null  object
 4   user_score    22264 non-null  object
 5   developer     22264 non-null  object
 6   publisher     22264 non-null  object
 7   genre         22264 non-null  object
dtypes: object(8)
memory usage: 1.4+ MB


### Examine column data types

In [ ]:
def count_types(col):
    type_counts = df_clean[col].apply(lambda x: type(x).__name__).value_counts()
    print(f"Data type counts per column '{col}':")
    print(type_counts)

count_types('name')
count_types('platform')
count_types('release_date')
count_types('metascore')
count_types('user_score')
count_types('developer')
count_types('publisher')
count_types('genre')

Conteo de tipos en la columna 'name':
name
str    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'platform':
platform
str    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'release_date':
release_date
Timestamp    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'metascore':
metascore
float    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'user_score':
user_score
float    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'developer':
developer
str    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'publisher':
publisher
list    22264
Name: count, dtype: int64
Conteo de tipos en la columna 'genre':
genre
str    22264
Name: count, dtype: int64


### Observe the clean dataset

In [32]:
df_clean.head(15)

,name,platform,release_date,metascore,user_score,developer,publisher,genre
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,1998-11-23,99.0,9.1,Nintendo,"[Nintendo, Gradiente]",['Open-World Action']
1,SoulCalibur,Dreamcast,1999-09-08,98.0,7.6,Namco,[Namco],['3D Fighting']
2,SoulCalibur,iOS (iPhone/iPad),1999-09-08,73.0,7.8,Namco,[Namco],['3D Fighting']
3,SoulCalibur,Xbox 360,1999-09-08,79.0,7.3,Namco,[Namco],['3D Fighting']
4,Grand Theft Auto IV,PlayStation 3,2008-04-29,98.0,8.0,Rockstar North,"[Rockstar Games, Capcom]",['Open-World Action']
5,Grand Theft Auto IV,Xbox 360,2008-04-29,98.0,8.3,Rockstar North,"[Rockstar Games, Capcom]",['Open-World Action']
6,Grand Theft Auto IV,PC,2008-04-29,90.0,7.8,Rockstar North,"[Rockstar Games, Capcom]",['Open-World Action']
7,Super Mario Galaxy,Wii,2007-11-12,97.0,9.1,Nintendo,[Nintendo],['3D Platformer']
8,Super Mario Galaxy 2,Wii,2010-05-23,97.0,9.0,Nintendo EAD Tokyo,"[Nintendo, iQue]",['3D Platformer']
9,The Legend of Zelda: Breath of the Wild,Wii U,2017-03-03,96.0,8.5,Nintendo,[Nintendo],['Open-World Action']


In [33]:
df_clean.tail(15)

,name,platform,release_date,metascore,user_score,developer,publisher,genre
22249,Fast & Furious: Showdown,Xbox 360,2013-05-21,22.0,1.5,Firebrand Games,[Activision],['Auto Racing']
22250,Drake of the 99 Dragons,Xbox,2003-11-03,22.0,2.0,Idol FX,[Majesco],['Third Person Shooter']
22251,Afro Samurai 2: Revenge of Kuma Volume One,PlayStation 4,2015-09-22,21.0,3.0,Versus Evil,[Versus Evil],"[""3D Beat-'Em-Up""]"
22252,Infestation: Survivor Stories (The War Z),PC,2012-10-15,20.0,1.7,Fredaikis AB,"[Arktos Entertainment, OP Productions LLC]",['MMORPG']
22253,Deal or No Deal,DS,2007-07-23,20.0,3.8,Artefacts Studio,"[Destination Software, Mindscape, Koch Media]",['Trivia']
22254,Deal or No Deal,Game Boy Advance,2007-07-23,56.0,5.0,Artefacts Studio,"[Destination Software, Mindscape, Koch Media]",['Trivia']
22255,Alone in the Dark: Illumination,PC,2015-06-11,19.0,1.3,Pure FPS,[Atari SA],['Survival']
22256,Ride to Hell: Retribution,PC,2013-06-25,16.0,1.4,Eutechnyx,[Deep Silver],['Action Adventure']
22257,Ride to Hell: Retribution,PlayStation 3,2013-06-25,13.0,1.1,Eutechnyx,[Deep Silver],['Action Adventure']
22258,Ride to Hell: Retribution,Xbox 360,2013-06-25,19.0,1.3,Eutechnyx,[Deep Silver],['Action Adventure']
